In [1]:
import glob
import os
from patchify import patchify
import cv2

In [2]:
def removing(folder1, folder2):
    png_files=[file.replace('.png','')for file in os.listdir(folder1) if file.endswith('.png')]

    tif_files = [file.replace('_seed_mask.tif', '').replace('_root_mask.tif', '').replace('_occluded_root_mask.tif', '').replace('_shoot_mask.tif', '')
                 for file in os.listdir(folder2) if file.endswith(('_seed_mask.tif','_root_mask.tif','_occluded_root_mask.tif','_shoot_mask.tif'))]
    files_to_remove=set(png_files)- set(tif_files)
    for file in files_to_remove:
        file_path= os.path.join(folder1, file+'.png')
        os.remove(file_path)
removing('Dataset/test', 'Dataset/masks')
removing('Dataset/train', 'Dataset/masks')

In [3]:
def crop_image(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY +cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    coordinates=[x,y,w,h]
    cropped_image = image[y:y + h, x:x + h]
    
    return cropped_image, coordinates

In [4]:
def padder(image, patch_size):
    h = image.shape[0]
    w = image.shape[1]
    height_padding = ((h // patch_size) + 1) * patch_size - h
    width_padding = ((w // patch_size) + 1) * patch_size - w
    top_padding = int(height_padding / 2)
    bottom_padding = height_padding - top_padding

    left_padding = int(width_padding / 2)
    right_padding = width_padding - left_padding

    padded_image = cv2.copyMakeBorder(image, top_padding, bottom_padding, left_padding, right_padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    return padded_image

In [5]:
def create_and_save_patches(dataset_type, patch_size,mask_type, mask_name):
     # Get all image names in train_images

    for image_path in sorted(glob.glob(f'Dataset/{dataset_type}/*.png')):
        mask_path = image_path.replace(dataset_type, 'masks')
        mask_path = mask_path.replace('.png', mask_type)

        image=cv2.imread(image_path)
        mask = cv2.imread(mask_path, 0)
        image, coordinates =crop_image(image)
        mask = mask[coordinates[1]:coordinates[1] + coordinates[3], coordinates[0]:coordinates[0] + coordinates[3]]

        image=padder(image,patch_size)
        mask=padder(mask,patch_size)
        patches_img=patchify(image,(patch_size,patch_size,3), step=patch_size)
        patches_img=patches_img.reshape(-1,patch_size,patch_size,3)
            
        patches_ms=patchify(mask,(patch_size,patch_size), step=patch_size)
        patches_ms=patches_ms.reshape(-1,patch_size,patch_size,1)
        
        image_patch_path = image_path.replace(f'Dataset/{dataset_type}', f'Dataset_patched/{dataset_type}_images/{dataset_type}')
        for i, patch in enumerate(patches_img):
            image_patch_path_numbered = f'{image_patch_path[:-4]}_{i}.png'
            cv2.imwrite(image_patch_path_numbered, patch)

        mask_patch_path = mask_path.replace('Dataset/masks', f'Dataset_patched/{dataset_type}_masks/{mask_name}')
        for i, patch in enumerate(patches_ms):
            mask_patch_path_numbered = f'{mask_patch_path[:-4]}_{i}.tif'
            cv2.imwrite(mask_patch_path_numbered, patch)
            

In [6]:
create_and_save_patches(dataset_type='train', patch_size=256, mask_type='_root_mask.tif',mask_name='root')

In [10]:
create_and_save_patches(dataset_type='train', patch_size=256, mask_type='_occluded_root_mask.tif',mask_name='occluded_root')

In [8]:
create_and_save_patches(dataset_type='train', patch_size=256, mask_type='_seed_mask.tif',mask_name='seed')

In [9]:
create_and_save_patches(dataset_type='train', patch_size=256, mask_type='_shoot_mask.tif',mask_name='shoot')

In [7]:
create_and_save_patches(dataset_type='test', patch_size=256, mask_type='_root_mask.tif',mask_name='root')

In [6]:
create_and_save_patches(dataset_type='test', patch_size=256, mask_type='_occluded_root_mask.tif',mask_name='occluded_root')

In [7]:
create_and_save_patches(dataset_type='test', patch_size=256, mask_type='_seed_mask.tif',mask_name='seed')

In [13]:
create_and_save_patches(dataset_type='test', patch_size=256, mask_type='_shoot_mask.tif',mask_name='shoot')